<a href="https://colab.research.google.com/github/nkrao8506/learn/blob/main/Amazon_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
bittlingmayer_amazonreviews_path = kagglehub.dataset_download('bittlingmayer/amazonreviews')

print('Data source import complete.')


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/test.ft.txt.bz2
/kaggle/input/amazonreviews/train.ft.txt.bz2
/kaggle/input/amazonreviews/.nfs0000000014abac060000016a


In [ ]:
import bz2
import pandas as pd

In [ ]:
NUMBER_OF_LINES = 6000
data = {}

In [ ]:
for i, line in enumerate(bz2.open("/kaggle/input/amazonreviews/train.ft.txt.bz2", "rt", encoding="utf8")):

    if i == NUMBER_OF_LINES:
        break

    # label 1 is negativ and label 2 is positive
    label = 0 if line[:10] == "__label__1" else 1
    text = line[10:]

    localResult = {
        "label": label,
        "text": text
    }

    data[i] = localResult


df = pd.DataFrame(data).T
df = df.reset_index().rename(columns= {"index": "Id"})

In [ ]:
test_data = {}

for i, line in enumerate(bz2.open("/kaggle/input/amazonreviews/test.ft.txt.bz2", "rt", encoding="utf8")):

    if i == NUMBER_OF_LINES:
        break

    # label 1 is negativ and label 2 is positive
    label = 0 if line[:10] == "__label__1" else 1
    text = line[10:]

    localResult = {
        "label": label,
        "text": text
    }

    test_data[i] = localResult


test_df = pd.DataFrame(test_data).T
test_df = test_df.reset_index().rename(columns= {"index": "Id"})

In [ ]:
df.head()

,Id,label,text
0,0,1,Stuning even for the non-gamer: This sound tr...
1,1,1,The best soundtrack ever to anything.: I'm re...
2,2,1,Amazing!: This soundtrack is my favorite musi...
3,3,1,Excellent Soundtrack: I truly like this sound...
4,4,1,"Remember, Pull Your Jaw Off The Floor After H..."


In [ ]:
df["label"].value_counts()

,count
label,
0,3178
1,2822


In [ ]:
test_df.head()

,Id,label,text
0,0,1,Great CD: My lovely Pat has one of the GREAT ...
1,1,1,One of the best game music soundtracks - for ...
2,2,0,Batteries died within a year ...: I bought th...
3,3,1,"works fine, but Maha Energy is better: Check ..."
4,4,1,Great for the non-audiophile: Reviewed quite ...


In [ ]:
test_df["label"].value_counts()

,count
label,
1,3096
0,2904


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pickle
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import io
import base64
from bs4 import BeautifulSoup
import requests

In [ ]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [ ]:
# Simulate Neutral labels (heuristic-based)
def assign_neutral_labels(df):
    neutral_keywords = ['okay', 'average', 'fine', 'meh', 'normal']
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    df['is_neutral'] = df.apply(
        lambda row: 2 if (row['word_count'] < 10 or any(kw in row['text'].lower() for kw in neutral_keywords)) else row['label'],
        axis=1
    )
    return df

In [ ]:
# Load and prepare data
# df = pd.read_csv('/content/amazon_reviews.csv')
df['text'] = df['text'].apply(preprocess_text)
df = assign_neutral_labels(df)
df['label'] = df['is_neutral']  # Use 0 (Positive), 1 (Negative), 2 (Neutral)
df.drop("label", axis=1, inplace=True)
df.drop("Id", axis=1, inplace=True)

In [ ]:
# Split data
X = df['text']
y = df['is_neutral']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Vectorize
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# Hyperparameter tuning for Logistic Regression
lg_model = LogisticRegression(multi_class='multinomial')
param_dist = {
    'C': np.logspace(-3, 2, 6),
    'penalty': ['l2'],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [200, 500]
}
random_search = RandomizedSearchCV(lg_model, param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train_vectorized, y_train)

# Best model
best_lg_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)
print("Test Accuracy:", accuracy_score(y_test, best_lg_model.predict(X_test_vectorized)))
print(classification_report(y_test, best_lg_model.predict(X_test_vectorized)))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best Parameters: {'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 200, 'C': np.float64(10.0)}
Test Accuracy: 0.8258333333333333
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       598
           1       0.81      0.84      0.83       529
           2       0.96      0.37      0.53        73

    accuracy                           0.83      1200
   macro avg       0.87      0.69      0.74      1200
weighted avg       0.83      0.83      0.82      1200



In [ ]:
# # Save model and vectorizer
# with open('lg_model.pkl', 'wb') as f:
#     pickle.dump(best_lg_model, f)
# with open('vectorizer.pkl', 'wb') as f:
#     pickle.dump(vectorizer, f)

# Function to scrape Amazon reviews
def scrape_amazon_reviews(url, max_reviews=50):
    headers = {'User-Agent': 'Mozilla/5.0'}
    reviews = []
    page = 1
    while len(reviews) < max_reviews:
        try:
            response = requests.get(f"{url}&pageNumber={page}", headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            review_elements = soup.find_all('span', {'data-hook': 'review-body'})
            if not review_elements:
                break
            for review in review_elements:
                text = review.get_text().strip()
                reviews.append(preprocess_text(text))
                if len(reviews) >= max_reviews:
                    break
            page += 1
        except Exception as e:
            print(f"Error scraping page {page}: {e}")
            break
    return reviews

# Function to generate word cloud
def generate_wordcloud(texts):
    combined_text = ' '.join(texts)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(combined_text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode('utf-8')
    plt.close()
    return img_str

In [ ]:
df.head()

,text,word_count,is_neutral
0,stuning even for the nongamer this sound trac...,80,1
1,the best soundtrack ever to anything im readi...,97,1
2,amazing this soundtrack is my favorite music ...,127,1
3,excellent soundtrack i truly like this soundt...,116,1
4,remember pull your jaw off the floor after he...,87,1


In [ ]:
df["is_neutral"].value_counts()

,count
is_neutral,
0,2967
1,2629
2,404


In [ ]:
df.head()

,text,word_count,is_neutral
0,stuning even for the nongamer this sound trac...,80,1
1,the best soundtrack ever to anything im readi...,97,1
2,amazing this soundtrack is my favorite music ...,127,1
3,excellent soundtrack i truly like this soundt...,116,1
4,remember pull your jaw off the floor after he...,87,1


In [ ]:
max_words = max(df["word_count"])
print(max_words)

198


In [ ]:
lg_model.fit(X_train_vectorized, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='multinomial')

In [ ]:
X_test_vectorized

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 58044 stored elements and shape (1200, 5000)>

In [ ]:
preds = lg_model.predict(X_test_vectorized)

In [ ]:
import plotly.express as px

positive_predictions = np.sum(preds == 1)
negative_predictions = np.sum(preds == 0)
neutral_predictions = np.sum(preds == 2)

fig = px.bar(
    x=['Positive', 'Negative', 'Neutral'],
    y=[positive_predictions, negative_predictions, neutral_predictions],
    color=['Positive', 'Negative', 'Neutral'],
    color_discrete_map={'Positive': 'green', 'Negative': 'red', 'Neutral': 'blue'},
    labels={'x': 'Sentiment', 'y': 'Count'},
    title='Predicted Sentiment Distribution',
    width = 800,
    height = 900
)
fig.show()

In [ ]:
def predict(text, model, vectorizer):
    text = preprocess_text(text)
    text_vectorized = vectorizer.transform([text])
    prediction = model.predict(text_vectorized)
    return prediction

In [ ]:
predict(text="This product performs as expected. It meets the basic requirements for its function, though it doesn't particularly excel or disappoint. The quality is acceptable, and it's reasonably priced. Overall, it's an average product that serves its purpose adequately without standing out.", model=lg_model, vectorizer=vectorizer)

array([0])

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
}

xgb = XGBClassifier(n_jobs=-1, use_label_encoder=False, eval_metric='logloss')

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,               # Number of parameter settings sampled
    scoring='accuracy',      # Or another metric suitable for your task
    cv=5,                    # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the search
random_search.fit(X_train_vectorized, y_train)

# Best parameters and estimator
print("Best parameters found: ", random_search.best_params_)
print("Best estimator: ", random_search.best_estimator_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:12:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 7, 'learning_rate': 0.2, 'gamma': 0.3, 'colsample_bytree': 0.6}
Best estimator:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.3, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=3, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, objective='multi:softprob', ...)


In [ ]:
xgbclf_preds = xgb.predict(X_test_vectorized)

In [ ]:
import plotly.express as px

positive_predictions = np.sum(xgbclf_preds == 1)
negative_predictions = np.sum(xgbclf_preds == 0)
neutral_predictions = np.sum(xgbclf_preds == 2)

fig = px.bar(
    x=['Positive', 'Negative', 'Neutral'],
    y=[positive_predictions, negative_predictions, neutral_predictions],
    color=['Positive', 'Negative', 'Neutral'],
    color_discrete_map={'Positive': 'green', 'Negative': 'red', 'Neutral': 'blue'},
    labels={'x': 'Sentiment', 'y': 'Count'},
    title='Predicted Sentiment Distribution',
    width = 800,
    height = 900
)
fig.show()